# Solvers benchmark on the `Wine Quality` data set

The data set can be downloaded from <http://archive.ics.uci.edu/ml/datasets/Wine+Quality>.

The solvers (except our `path_solver`) were coded by Larsson et al. for the benchmark of their paper *Coordinate Descent for SLOPE*; their code is available at <https://github.com/jolars/slopecd> and needs to be installed to run the simulations here.

In [1]:
import sys
import numpy as np
import pandas as pd
from timeit import repeat

sys.path.append('../..')
from modules import path_solver, dual_sorted_L1_norm as dual_norm, PD_gap
from slope.solvers import prox_grad, admm, hybrid_cd #, newt_alm, oracle_cd

In [2]:
# Import of the data set
wine = pd.read_csv('../datasets/winequality-red.csv', sep=';')
data = wine.drop(columns=['quality'])
target = wine['quality']

In [3]:
# Setting (and data standardization)
X = data.to_numpy(dtype=float)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0, ddof=0)
X = (X - X_mean) / X_std

y = target.to_numpy(dtype=float)
y_mean = y.mean()
y = y - y_mean

# Lambda = np.linspace(4,1,X.shape[-1],dtype=float)
Lambda = np.sqrt(range(1,12))-np.sqrt(range(0,11))

In [4]:
# # Numba compilation
# _ = path_solver(X, y, Lambda, k_max=0., rtol_pattern=1e-6, atol_pattern = 1e-6, rtol_gamma=1e-6, split_max=1e1, log=0)
# _ = hybrid_cd(X, y, Lambda, fit_intercept=False, tol=1e-1)
# # _ = oracle_cd(X, y, Lambda, fit_intercept=False, tol=1e-1)
# _ = admm(X, y, Lambda, fit_intercept=False, rho=100, adaptive_rho=False, tol=1e-1)
# # _ = newt_alm(X, y, Lambda, fit_intercept=False, tol=1e-1)
# _ = prox_grad(X, y, Lambda, fit_intercept=False, tol=1e-1)

In [15]:
# Check-up and numba compilation
ratio = 0.1; gamma = ratio*dual_norm(X.T@y, Lambda)
nb_loops = 1; nb_runs = 1

# FISTA
sol, intercept, primals, gaps, times = prox_grad(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, fista=True, tol=1e-12/X.shape[0])
time_fista =np.mean(repeat('prox_grad(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, fista=True, tol=1e-12/X.shape[0])',
                   repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
print(f'FISTA: {time_fista:.2e}s')
print(sol, X.shape[0]*primals[-1], X.shape[0]*gaps[-1],'\n')

# Anderson PGD
sol, intercept, primals, gaps, times = prox_grad(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, anderson=True, tol=1e-12/X.shape[0])
time_pgd =np.mean(repeat('prox_grad(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, anderson=True, tol=1e-12/X.shape[0])',
                   repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
print(f'Anderson PGD: {time_pgd:.2e}s')
print(sol, X.shape[0]*primals[-1], X.shape[0]*gaps[-1],'\n')

# ADMM (with rho=100)
sol, intercept, primals, gaps, times = admm(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, rho=100, adaptive_rho=False, tol=1e-12/X.shape[0])
time_admm =np.mean(repeat('admm(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, rho=100, adaptive_rho=False, tol=1e-12/X.shape[0])',
                   repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
print(f'ADMM: {time_admm:.2e}s')
print(sol, X.shape[0]*primals[-1], X.shape[0]*gaps[-1],'\n')

# hybrid CD
sol, intercept, primals, gaps, times, n_cluster = hybrid_cd(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, tol=1e-12/X.shape[0])
time_cd =np.mean(repeat('hybrid_cd(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, tol=1e-12/X.shape[0])',
                   repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
print(f'hybrid CD: {time_cd:.2e}s')
print(sol, X.shape[0]*primals[-1], X.shape[0]*gaps[-1],'\n')

# path (our)
sol, (primal, gap), k = path_solver(X, y, gamma*Lambda, k_max=1e3, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=0)
time_path =np.mean(repeat('path_solver(X, y, gamma*Lambda, k_max=1e3, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=0)',
                   repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
print(f'path (our): {time_path:.2e}s')
print(sol, primal, gap, '\n')

# # oracle CD
# sol, intercept, primals, gaps, times = oracle_cd(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, tol=1e-12)
# print(sol, X.shape[0]*primals[-1], gaps[-1])
# time_oracle =np.mean(repeat('oracle_cd(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, tol=1e-12)',
#                    repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
# print(f'oracle CD: {time_oracle:.2e}s')

# # Newt-ALM
# sol, intercept, primals, gaps, times = newt_alm(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, tol=1e-12)
# print(sol, X.shape[0]*primals[-1], gaps[-1])
# time_newt_alm =np.mean(repeat('newt_alm(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, tol=1e-12)',
#                    repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
# print(f'Newt-ALM: {time_newt_alm:.2e}s')

FISTA: 5.02e-01s
[ 0.07328962 -0.17520925  0.          0.03055732 -0.08215734  0.01589062
 -0.08431084 -0.08431084 -0.014562    0.14920723  0.20882182] 374.52499980726094 9.76385639006594e-13 

Anderson PGD: 5.95e-02s
[ 0.07328962 -0.17520925  0.          0.03055732 -0.08215734  0.01589062
 -0.08431084 -0.08431084 -0.014562    0.14920723  0.20882182] 374.52499980726094 9.76385639006594e-13 

ADMM: 1.72e-02s
[ 7.32896188e-02 -1.75209246e-01  6.82586500e-14  3.05573223e-02
 -8.21573432e-02  1.58906203e-02 -8.43108421e-02 -8.43108421e-02
 -1.45619996e-02  1.49207231e-01  2.08821822e-01] 374.5249998072608 9.76385639006594e-13 

hybrid CD: 5.57e-03s
[ 0.07328962 -0.17520925  0.          0.03055732 -0.08215734  0.01589062
 -0.08431084 -0.08431084 -0.014562    0.14920723  0.20882182] 374.52499980726094 9.76385639006594e-13 

path (our): 2.46e-03s
[ 0.07328962 -0.17520925  0.          0.03055732 -0.08215734  0.01589062
 -0.08431084 -0.08431084 -0.014562    0.14920723  0.20882182] 374.524999807

In [9]:
# Benchmark
benchmark = pd.DataFrame({},index=['FISTA', 'Anderson PGD', 'ADMM (rho=100)', 'hybrid CD', 'path (our)'])
benchmark.columns.name='gamma / gamma_max'

In [16]:
nb_loops = 1000; nb_runs = 7
# for ratio in [0.5, 0.1, 0.02]:
gamma = ratio*dual_norm(X.T@y, Lambda)
time_fista =np.mean(repeat('prox_grad(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, fista=True, tol=1e-12/X.shape[0])',
                    repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
time_pgd =np.mean(repeat('prox_grad(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, anderson=True, tol=1e-12/X.shape[0])',
                    repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
time_admm =np.mean(repeat('admm(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, rho=100, adaptive_rho=False, tol=1e-12/X.shape[0])',
                    repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
time_cd =np.mean(repeat('hybrid_cd(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, tol=1e-12/X.shape[0])',
                    repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
time_path =np.mean(repeat('path_solver(X, y, gamma*Lambda, k_max=1e3, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=0)',
                    repeat=nb_runs, number=nb_loops, globals=globals()))/nb_loops
benchmark[f'{ratio}'] = [time_fista, time_pgd, time_admm, time_cd, time_path]
with pd.option_context('display.float_format', '{:,.2e}'.format):
    display(benchmark)   


gamma / gamma_max,0.5,0.1
FISTA,1.36e-02,4.47e-02
Anderson PGD,5.26e-03,7.02e-02
ADMM (rho=100),2.38e-02,7.18e-03
hybrid CD,2.39e-03,7.91e-03
path (our),6.58e-04,4.51e-03


In [17]:
benchmark.to_csv('../results/wine-quality_benchmark.csv', float_format='{:.2e}'.format, mode='a')

In [18]:
with open('../results/wine-quality_benchmark.txt', 'a') as f:
                f.write(benchmark.to_latex(float_format='{:.2e}'.format))